In [1]:
import json
import time
import sys
import pandas as pd
import numpy  as np
import scipy.sparse as sp
import pickle as pkl
import collections 
import gc

In [2]:
with open('../aps/index_item_map.pkl', 'rb') as f:
    data_map = pkl.load(f)

In [3]:
paper_id_title = data_map['paper_id_title']
author_id_name = data_map['author_id_name']
venue_id_name = data_map['venue_id_name']
keywords_id_name = data_map['keywords_id_name']
paper_title_id = data_map['paper_title_id']
author_name_id = data_map['author_name_id']
venue_name_id = data_map['venue_name_id']
keywords_name_id = data_map['keywords_name_id']
keywords_set = data_map['keywords_set']
venue_set = data_map['venue_set']

In [4]:
full_pair = pd.read_csv('../aps/whole_graph.csv')

In [5]:
def construct_index_from_one_df(df,item_id,flag):
    if flag == 'P':
        max_len = 100
    elif flag == 'A':
        max_len = 20
    elif flag == 'V':
        max_len = 1
    elif flag == 'K':
        max_len = 15
    dic = collections.defaultdict(lambda : [])
    keys = list(df['P'])
    values = list(df['O'])
    for i in  range(len(keys)):
        try:
            if len(dic[keys[i]])<max_len:
                index_id = item_id[values[i]]
                dic[keys[i]].append(index_id)
            cnt = cnt + 1
        except:
            pass
    for i in range(len(keys)):
        need_len = max_len-len(dic[keys[i]])
        for j in range(need_len):
            dic[keys[i]].append(-1)
    return dic


In [6]:
def construct_index_5_years(df,item_list,flag):
    dic_1 = construct_index_from_one_df(df,item_list[0],flag)
    dic_2 = construct_index_from_one_df(df,item_list[1],flag)
    dic_3 = construct_index_from_one_df(df,item_list[2],flag)
    dic_4 = construct_index_from_one_df(df,item_list[3],flag)
    dic_5 = construct_index_from_one_df(df,item_list[4],flag)
    return [dic_1,dic_2,dic_3,dic_4,dic_5]

In [7]:
def index_mapping_one(paper_lst,id_dic,num):
    xx = []
    yy = list(np.zeros(num))
    for paper in paper_lst:
        if len(id_dic[paper]) ==0:
            xx.append(yy)
        else:
            xx.append(id_dic[paper])
    return xx
#     return [id_dic[paper] for paper in paper_lst]
def index_mapping_five(paper_lst,id_dic_list,num):
    return np.array([index_mapping_one(paper_lst,id_dic,num) for id_dic in id_dic_list])

In [8]:
with open('../aps/individual_graph_1995.pkl','rb') as f:
    graph_1995 = pkl.load(f)
item_id_1995 = graph_1995['item_id']
with open('../aps/individual_graph_1996.pkl','rb') as f:
    graph_1996 = pkl.load(f)
item_id_1996 = graph_1996['item_id']
with open('../aps/individual_graph_1997.pkl','rb') as f:
    graph_1997 = pkl.load(f)
item_id_1997 = graph_1997['item_id']
with open('../aps/individual_graph_1998.pkl','rb') as f:
    graph_1998 = pkl.load(f)
item_id_1998 = graph_1998['item_id']
with open('../aps/individual_graph_1999.pkl','rb') as f:
    graph_1999 = pkl.load(f)
item_id_1999 = graph_1999['item_id']
with open('../aps/individual_graph_2000.pkl','rb') as f:
    graph_2000 = pkl.load(f)
item_id_2000 = graph_2000['item_id']
with open('../aps/individual_graph_2001.pkl','rb') as f:
    graph_2001 = pkl.load(f)
item_id_2001 = graph_2001['item_id']
with open('../aps/individual_graph_2002.pkl','rb') as f:
    graph_2002 = pkl.load(f)
item_id_2002 = graph_2002['item_id']
with open('../aps/individual_graph_2003.pkl','rb') as f:
    graph_2003 = pkl.load(f)
item_id_2003 = graph_2003['item_id']
with open('../aps/individual_graph_2004.pkl','rb') as f:
    graph_2004 = pkl.load(f)
item_id_2004 = graph_2004['item_id']

In [9]:
year  = 2000
t = time.time()
P1Y_2005 = full_pair[full_pair.O==year]
all_paper_2005 = pd.DataFrame({"P":list(set(P1Y_2005['P']))})
full_pair_2005 = pd.merge(all_paper_2005,full_pair,how="left")
full_pair_2005_P1P = full_pair_2005[full_pair_2005.type=='P1P']
full_pair_2005_P1A = full_pair_2005[full_pair_2005.type=='P1A']
full_pair_2005_P1V = full_pair_2005[full_pair_2005.type=='P1V']
full_pair_2005_P1K = full_pair_2005[full_pair_2005.type=='P1K']
graph_lookip_list = [item_id_1995,item_id_1996,item_id_1997,item_id_1998,item_id_1999]
P2005_1_P1P_list = construct_index_5_years(full_pair_2005_P1P,graph_lookip_list,'P')
P2005_1_P1A_list = construct_index_5_years(full_pair_2005_P1A,graph_lookip_list,'A')
P2005_1_P1V_list = construct_index_5_years(full_pair_2005_P1V,graph_lookip_list,'V')
P2005_1_P1K_list = construct_index_5_years(full_pair_2005_P1K,graph_lookip_list,'K')

cumulative_labels = pd.read_csv('../aps/log_labels'+str(year)+'.txt')
P2005_label_cumu = list(cumulative_labels['P'])
P2005_1_P1P_list = index_mapping_five(P2005_label_cumu,P2005_1_P1P_list,100)
P2005_1_P1A_list = index_mapping_five(P2005_label_cumu,P2005_1_P1A_list,20)
P2005_1_P1V_list = index_mapping_five(P2005_label_cumu,P2005_1_P1V_list,1)
P2005_1_P1K_list = index_mapping_five(P2005_label_cumu,P2005_1_P1K_list,15)
print ((P2005_1_P1P_list).shape)
print ((P2005_1_P1A_list).shape)
print ((P2005_1_P1V_list).shape)
print ((P2005_1_P1K_list).shape)
with open('../aps/index_'+str(year)+'.pkl','wb') as f:
    pkl.dump([P2005_1_P1P_list,P2005_1_P1A_list,P2005_1_P1V_list,P2005_1_P1K_list],f,0)
time.time()-t

(5, 14643, 100)
(5, 14643, 20)
(5, 14643, 1)
(5, 14643, 15)


4.996817111968994

In [10]:
P2005_1_P1V_list[-1].reshape(-1)

array([102870, 102870, 102870, ..., 102877, 102877, 102877])

In [11]:
year  = 2001
t = time.time()
P1Y_2005 = full_pair[full_pair.O==year]
all_paper_2005 = pd.DataFrame({"P":list(set(P1Y_2005['P']))})
full_pair_2005 = pd.merge(all_paper_2005,full_pair,how="left")
full_pair_2005_P1P = full_pair_2005[full_pair_2005.type=='P1P']
full_pair_2005_P1A = full_pair_2005[full_pair_2005.type=='P1A']
full_pair_2005_P1V = full_pair_2005[full_pair_2005.type=='P1V']
full_pair_2005_P1K = full_pair_2005[full_pair_2005.type=='P1K']
graph_lookip_list = [item_id_1996,item_id_1997,item_id_1998,item_id_1999,item_id_2000]
P2005_1_P1P_list = construct_index_5_years(full_pair_2005_P1P,graph_lookip_list,'P')
P2005_1_P1A_list = construct_index_5_years(full_pair_2005_P1A,graph_lookip_list,'A')
P2005_1_P1V_list = construct_index_5_years(full_pair_2005_P1V,graph_lookip_list,'V')
P2005_1_P1K_list = construct_index_5_years(full_pair_2005_P1K,graph_lookip_list,'K')

cumulative_labels = pd.read_csv('../aps/log_labels'+str(year)+'.txt')
P2005_label_cumu = list(cumulative_labels['P'])
P2005_1_P1P_list = index_mapping_five(P2005_label_cumu,P2005_1_P1P_list,100)
P2005_1_P1A_list = index_mapping_five(P2005_label_cumu,P2005_1_P1A_list,20)
P2005_1_P1V_list = index_mapping_five(P2005_label_cumu,P2005_1_P1V_list,1)
P2005_1_P1K_list = index_mapping_five(P2005_label_cumu,P2005_1_P1K_list,15)
print ((P2005_1_P1P_list).shape)
print ((P2005_1_P1A_list).shape)
print ((P2005_1_P1V_list).shape)
print ((P2005_1_P1K_list).shape)
with open('../aps/index_'+str(year)+'.pkl','wb') as f:
    pkl.dump([P2005_1_P1P_list,P2005_1_P1A_list,P2005_1_P1V_list,P2005_1_P1K_list],f,0)
time.time()-t

(5, 14936, 100)
(5, 14936, 20)
(5, 14936, 1)
(5, 14936, 15)


4.463131427764893

In [12]:
year  = 2005
t = time.time()
P1Y_2005 = full_pair[full_pair.O==year]
all_paper_2005 = pd.DataFrame({"P":list(set(P1Y_2005['P']))})
full_pair_2005 = pd.merge(all_paper_2005,full_pair,how="left")
full_pair_2005_P1P = full_pair_2005[full_pair_2005.type=='P1P']
full_pair_2005_P1A = full_pair_2005[full_pair_2005.type=='P1A']
full_pair_2005_P1V = full_pair_2005[full_pair_2005.type=='P1V']
full_pair_2005_P1K = full_pair_2005[full_pair_2005.type=='P1K']
graph_lookip_list = [item_id_2000,item_id_2001,item_id_2002,item_id_2003,item_id_2004]
P2005_1_P1P_list = construct_index_5_years(full_pair_2005_P1P,graph_lookip_list,'P')
P2005_1_P1A_list = construct_index_5_years(full_pair_2005_P1A,graph_lookip_list,'A')
P2005_1_P1V_list = construct_index_5_years(full_pair_2005_P1V,graph_lookip_list,'V')
P2005_1_P1K_list = construct_index_5_years(full_pair_2005_P1K,graph_lookip_list,'K')

cumulative_labels = pd.read_csv('../aps/log_labels'+str(year)+'.txt')
P2005_label_cumu = list(cumulative_labels['P'])
P2005_1_P1P_list = index_mapping_five(P2005_label_cumu,P2005_1_P1P_list,100)
P2005_1_P1A_list = index_mapping_five(P2005_label_cumu,P2005_1_P1A_list,20)
P2005_1_P1V_list = index_mapping_five(P2005_label_cumu,P2005_1_P1V_list,1)
P2005_1_P1K_list = index_mapping_five(P2005_label_cumu,P2005_1_P1K_list,15)
print ((P2005_1_P1P_list).shape)
print ((P2005_1_P1A_list).shape)
print ((P2005_1_P1V_list).shape)
print ((P2005_1_P1K_list).shape)
with open('../aps/index_'+str(year)+'.pkl','wb') as f:
    pkl.dump([P2005_1_P1P_list,P2005_1_P1A_list,P2005_1_P1V_list,P2005_1_P1K_list],f,0)
time.time()-t

(5, 17953, 100)
(5, 17953, 20)
(5, 17953, 1)
(5, 17953, 15)


5.442184686660767